In [ ]:
# source:
# https://blog.quantinsti.com/decision-tree/

# https://blog.quantinsti.com/parabolic-sar/

In [ ]:
!which python3

In [ ]:
import talib as ta

In [ ]:
import pandas as pd

#suppress 'SettingWithCopy' warning
pd.set_option('mode.chained_assignment', None)

In [ ]:
#!pip install pandas_datareader
#!pip3 install seaborn
import seaborn as sns

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# ___library_import_statements___
import pandas as pd

# for pandas_datareader, otherwise it might have issues, sometimes there is some version mismatch
pd.core.common.is_list_like = pd.api.types.is_list_like

# make pandas to print dataframes nicely
pd.set_option('expand_frame_repr', False)  

import pandas_datareader.data as web
import numpy as np
import matplotlib.pyplot as plt
import datetime
import time

#newest yahoo API 
import yfinance as yahoo_finance

#optional 
#yahoo_finance.pdr_override()

%matplotlib inline

In [ ]:
import talib as ta
import numpy as np

import matplotlib.pyplot as plt

# was giving me some warnings
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [ ]:
# ___variables___
ticker = 'AAPL'
#ticker = 'TSLA'
#ticker = 'FB'
#ticker = 'MSFT'
#ticker = 'NFLX'
#ticker = 'GOOGL'
#ticker = 'BIDU'
#ticker = 'AMZN'
ticker = 'IBM'

start_time = datetime.datetime(1980, 1, 1)
#end_time = datetime.datetime(2019, 1, 20)
end_time = datetime.datetime.now().date().isoformat()         # today

# --- FUNCTION DEFINITIONS ---

In [ ]:

def get_data(ticker):
    # yahoo gives only daily historical data
    connected = False
    while not connected:
        try:
            ticker_df = web.get_data_yahoo(ticker, start=start_time, end=end_time)
            connected = True
            print('connected to yahoo')
        except Exception as e:
            print("type error: " + str(e))
            time.sleep( 5 )
            pass   

    # use numerical integer index instead of date    
    ticker_df = ticker_df.reset_index()
    print(ticker_df.head(5))
    return ticker_df

In [ ]:
#df = get_data(ticker)

In [ ]:
#df.head()
#df.tail()
#df.shape

In [ ]:
def compute_technical_indicators(df):
    df['EMA10'] = ta.EMA(df['Adj Close'].values, timeperiod=10)
    df['EMA20'] = ta.EMA(df['Adj Close'].values, timeperiod=10)
    df['EMA30'] = ta.EMA(df['Adj Close'].values, timeperiod=30)
    df['EMA40'] = ta.EMA(df['Adj Close'].values, timeperiod=40)
    df['EMA50'] = ta.EMA(df['Adj Close'].values, timeperiod=50)
    df['EMA100'] = ta.EMA(df['Adj Close'].values, timeperiod=100)
    df['EMA150'] = ta.EMA(df['Adj Close'].values, timeperiod=150)
    df['EMA200'] = ta.EMA(df['Adj Close'].values, timeperiod=200)

    df['upperBB'], df['middleBB'], df['lowerBB'] = ta.BBANDS(df['Adj Close'].values, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)

    df['SAR'] = ta.SAR(df['High'].values, df['Low'].values, acceleration=0.02, maximum=0.2)
    
    
    
    
    df['ATR'] = ta.ATR(df['High'].values, df['Low'].values, df['Adj Close'].values, timeperiod=14)
    df['ADX'] = ta.ADX(df['High'].values, df['Low'].values, df['Adj Close'].values, timeperiod=14)
    df['RSI'] = ta.RSI(df['Adj Close'].values, timeperiod=14)

    macd, macdsignal, macdhist = ta.MACD(df['Adj Close'].values, fastperiod=12, slowperiod=26, signalperiod=9)
    df['MACD'] = macd
    df['MACDsignal'] = macdsignal
    df.tail()

    return df

In [ ]:
#df = compute_technical_indicators(df)

In [ ]:
def compute_features(df):
    # computes features for forest decisions
    df['aboveEMA10'] = np.where(df['Adj Close'] > df['EMA10'], 1, -1)
    df['aboveEMA20'] = np.where(df['Adj Close'] > df['EMA20'], 1, -1)
    df['aboveEMA30'] = np.where(df['Adj Close'] > df['EMA30'], 1, -1)
    df['aboveEMA40'] = np.where(df['Adj Close'] > df['EMA40'], 1, -1)


    df['aboveEMA50'] = np.where(df['Adj Close'] > df['EMA50'], 1, -1)
    df['aboveEMA100'] = np.where(df['Adj Close'] > df['EMA100'], 1, -1)
    df['aboveEMA150'] = np.where(df['Adj Close'] > df['EMA150'], 1, -1)
    df['aboveEMA200'] = np.where(df['Adj Close'] > df['EMA200'], 1, -1)

    df['aboveUpperBB'] = np.where(df['Adj Close'] > df['upperBB'], 1, -1)
    df['belowLowerBB'] = np.where(df['Adj Close'] < df['lowerBB'], 1, -1)
    
    df['aboveSAR'] = np.where(df['Adj Close'] > df['SAR'], 1, -1)
   
    df['oversoldRSI'] = np.where(df['RSI'] < 30, 1, -1)
    df['overboughtRSI'] = np.where(df['RSI'] > 70, 1, -1)

    df['ClgtEMA10'] = np.where(df['Adj Close'] > df['EMA10'], 1, -1)
    df['EMA10gtEMA30'] = np.where(df['EMA10'] > df['EMA30'], 1, -1)
    df['MACDSIGgtMACD'] = np.where(df['MACDsignal'] > df['MACD'], 1, -1)

    # delete below line
    df['Buy'] = np.where(df['Adj Close'] > df['EMA50'], 1, -1)

    # very important - cleanup NaN values, otherwise prediction does not work
    df=df.fillna(0).copy()
    
    df.tail()

    return df

In [ ]:
#df = compute_features(df)

In [ ]:
def plot_train_data(df):
    # plot price
    plt.figure(figsize=(30,5))
    plt.plot(df.index, df['Adj Close'])
    #plt.title('Price chart (Adj Close) ' + str(ticker))
    plt.show()
    return None

In [ ]:
#plot_train_data(df)

## Target condition - criterion

In [ ]:
def define_target_condition(df):
    # my new target condition
    #df['target_cls'] = np.where(df['Adj Close'].shift(-20) > df.EMA200.shift(-20), 1, 0)

    # even newer my new target condition
    #df['target_cls'] = np.where(df['Adj Close'].shift(-20) > df.EMA100.shift(-20), 1, 0)

    # even even newer my new target condition
    df['target_cls'] = np.where(df['Adj Close'].shift(-50) > df.EMA150.shift(-50), 1, 0)

    # important
    df=df.fillna(0).copy()
    
    df.tail()
    
    return df

In [ ]:
#df=define_target_condition(df)

#### Train-Test split
#### Training part
#### Actual training part - it is just one line actually

#### partial fit should keep old weights and improve them with further training
turns out decision trees have no partial_fit option -  have to be trained in one go
but check random forests they look like they can learn in sequences


actually
You can set the 'warm_start' parameter to True in the model. This will ensure the retention of learning with previous learn using fit call.

Same model learning incrementally two times (train_X[:1], train_X[1:2]) after setting ' warm_start '

forest_model = RandomForestRegressor(warm_start=True)
forest_model.fit(train_X[:1],train_y[:1])
pred_y = forest_model.predict(val_X[:1])
mae = mean_absolute_error(pred_y,val_y[:1])
print("mae      :",mae)
print('pred_y :',pred_y)
forest_model.fit(train_X[1:2],train_y[1:2])
pred_y = forest_model.predict(val_X[1:2])
mae = mean_absolute_error(pred_y,val_y[1:2])
print("mae      :",mae)
print('pred_y :',pred_y)

In [ ]:
def splitting_and_training(df):
    # __predictors__
    #predictors_list = ['RSI', 'ClgtEMA10', 'EMA10gtEMA30', 'MACDSIGgtMACD']
    #predictors_list = ['oversoldRSI', 'overboughtRSI', 'aboveEMA50', 'aboveEMA100', 'aboveEMA150', 'aboveEMA200']
    predictors_list = ['aboveSAR','aboveUpperBB','belowLowerBB','RSI','oversoldRSI', 'overboughtRSI', 'ClgtEMA10', 'EMA10gtEMA30', 'MACDSIGgtMACD','aboveEMA10','aboveEMA20','aboveEMA30','aboveEMA40','aboveEMA50', 'aboveEMA100', 'aboveEMA150', 'aboveEMA200']

    #print(df[predictors_list])
    
    # __features__
    X = df[predictors_list].fillna(0)
    X.tail()

    # __targets__
    y_cls = df.target_cls.fillna(0)
    y_cls.tail(10)

    # __train test split__
    from sklearn.model_selection import train_test_split
    y=y_cls
    X_cls_train, X_cls_test, y_cls_train, y_cls_test = train_test_split(X, y, test_size=0.3, random_state=432, stratify=y)

    print (X_cls_train.shape, y_cls_train.shape)
    print (X_cls_test.shape, y_cls_test.shape)

    # __RANDOM FOREST __       - retrainable - warm_start
    from sklearn.ensemble import RandomForestClassifier

    #Create a Gaussian Classifier - incremental training - warm_start=True
    clf=RandomForestClassifier(n_estimators=50, criterion='gini', max_depth=6, min_samples_leaf=6, warm_start=True)

    # __ACTUAL TRAINING __
    clf = clf.fit(X_cls_train, y_cls_train)
    clf

    # __making accuracy report__
    # ideally should be getting better with each round
    y_cls_pred = clf.predict(X_cls_test)

    from sklearn.metrics import classification_report
    report = classification_report(y_cls_test, y_cls_pred)
    print(report)

    return clf

In [ ]:
# get trained classifier
#clf = splitting_and_training(df)

In [ ]:
#plot_train_data(df)

In [ ]:
#df

In [ ]:
def predict_timeseries(df, clf):
    # oversoldRSI 	oversoldRSI 	overboughtRSI 	aboveEMA50 	aboveEMA100 	aboveEMA150 	aboveEMA200

    for i in range(len(df)):
        #X_cls_valid = [[df['oversoldRSI'][i],df['overboughtRSI'][i],df['aboveEMA50'][i],df['aboveEMA100'][i],df['aboveEMA150'][i],df['aboveEMA200'][i]]]
        #X_cls_valid = [[df['aboveEMA10'][i],df['aboveEMA20'][i],df['aboveEMA30'][i],df['aboveEMA40'][i],df['aboveEMA50'][i],df['aboveEMA100'][i],df['aboveEMA150'][i],df['aboveEMA200'][i]]]

        X_cls_valid = [[df['aboveSAR'][i],df['aboveUpperBB'][i],df['belowLowerBB'][i],df['RSI'][i],df['oversoldRSI'][i],df['overboughtRSI'][i],df['ClgtEMA10'][i],df['EMA10gtEMA30'][i],df['MACDSIGgtMACD'][i],df['aboveEMA10'][i],df['aboveEMA20'][i],df['aboveEMA30'][i],df['aboveEMA40'][i],df['aboveEMA50'][i],df['aboveEMA100'][i],df['aboveEMA150'][i],df['aboveEMA200'][i]]]    

        y_cls_pred_valid = clf.predict(X_cls_valid)
        df['Buy'][i] = y_cls_pred_valid[0].copy()

    return df

In [ ]:
#df = predict_timeseries(df, clf)

In [ ]:
def plot_stock_prediction(df, ticker):
    # plot  values and significant levels
    plt.figure(figsize=(30,10))
    plt.title('Predictive model ' + str(ticker))
    plt.plot(df['Date'], df['Adj Close'], label='High', alpha=0.2)

    plt.plot(df['Date'], df['EMA10'], label='High', alpha=0.2)
    plt.plot(df['Date'], df['EMA20'], label='High', alpha=0.2)
    plt.plot(df['Date'], df['EMA30'], label='High', alpha=0.2)
    plt.plot(df['Date'], df['EMA40'], label='High', alpha=0.2)
    plt.plot(df['Date'], df['EMA50'], label='High', alpha=0.2)
    plt.plot(df['Date'], df['EMA100'], label='High', alpha=0.2)
    plt.plot(df['Date'], df['EMA150'], label='High', alpha=0.99)
    plt.plot(df['Date'], df['EMA200'], label='High', alpha=0.2)


    plt.scatter(df['Date'], df['Buy']*df['Adj Close'], label='Buy', marker='^', color='magenta', alpha=0.15)
    #lt.scatter(df.index, df['sell_sig'], label='Sell', marker='v')

    plt.legend()

    plt.show()

    return None   

# --- EXECUTION SEQUENCE ---

In [ ]:
for ticker in ['SPY', 'F', 'IBM', 'GE', 'AAPL', 'T', 'ADM', 'XOM', 'GM','MMM','KO','PEP','SO','GS']:

    df = get_data(ticker)
    plot_train_data(df)
    df = compute_technical_indicators(df)
    df = compute_features(df)
    df=define_target_condition(df)

    clf = splitting_and_training(df)

    df = predict_timeseries(df, clf)
    plot_stock_prediction(df, ticker)

# 
# --- LIVE PREDICTION ---
# 

In [ ]:
ticker='BP'

In [ ]:
new_df = get_data(ticker)

In [ ]:
new_df = compute_technical_indicators(new_df)

In [ ]:
new_df = compute_features(new_df)

In [ ]:
new_df=define_target_condition(new_df)

In [ ]:
new_df = predict_timeseries(new_df, clf)

In [ ]:
plot_stock_prediction(new_df, ticker)

In [ ]:
temp_df = new_df[-700:]

In [ ]:
plot_stock_prediction(temp_df, ticker)